In [ ]:
#Clasificación

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV, StratifiedShuffleSplit
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score,confusion_matrix, classification_report
import unidecode
from nltk.corpus import stopwords 

In [2]:
pd.set_option('display.max_columns', 100) # Para mostrar todas las columnas
# pd.set_option('display.max_rows', 100) # Para mostrar todas las filas

In [3]:
data = pd.read_csv("Data/movies_usa_en.csv", low_memory=False)
data.head()

,title,description_clean,genre_clean,romance,drama,history,biography,crime,horror,western,...,action,war,scifi,mystery,thriller,sport,musical,music,filmnoir,animation
0,Miss Jerry,the adventures of a female reporter in the 1890s,romance,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Cleopatra,the fabled queen of egypts affair with roman g...,drama history,0,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"From the Manger to the Cross; or, Jesus of Naz...",an account of the life of jesus christ based o...,biography drama,0,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Amore di madre,john howard payne at his most miserable point ...,drama,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Traffic in Souls,a woman with the aid of her police officer swe...,crime drama,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
data['genre_clean'].value_counts()


drama                       3679
comedy                      2679
comedy drama                1466
drama romance               1387
horror                      1321
                            ... 
family music romance           1
crime mystery war              1
drama action comedy            1
adventure family sport         1
romance thriller western       1
Name: genre_clean, Length: 962, dtype: int64

In [4]:
data.shape

(38075, 24)

In [5]:
data.columns

Index(['title', 'description_clean', 'genre_clean', 'romance', 'drama',
       'history', 'biography', 'crime', 'horror', 'western', 'fantasy',
       'comedy', 'family', 'adventure', 'action', 'war', 'scifi', 'mystery',
       'thriller', 'sport', 'musical', 'music', 'filmnoir', 'animation'],
      dtype='object')

In [6]:
data.drop(['romance', 'drama',
       'history', 'biography', 'crime', 'horror', 'western', 'fantasy',
       'comedy', 'family', 'adventure', 'action', 'war', 'scifi', 'mystery',
       'thriller', 'sport', 'musical', 'music', 'filmnoir', 'animation'], inplace=True, axis=1)

In [7]:
data.shape

(38075, 3)

In [8]:
data['Genre'] = data['genre_clean'].str.split(" ")

In [9]:
data.head()



,title,description_clean,genre_clean,Genre
0,Miss Jerry,the adventures of a female reporter in the 1890s,romance,[romance]
1,Cleopatra,the fabled queen of egypts affair with roman g...,drama history,"[drama, history]"
2,"From the Manger to the Cross; or, Jesus of Naz...",an account of the life of jesus christ based o...,biography drama,"[biography, drama]"
3,Amore di madre,john howard payne at his most miserable point ...,drama,[drama]
4,Traffic in Souls,a woman with the aid of her police officer swe...,crime drama,"[crime, drama]"


In [10]:
data['Genre_2'] = data['Genre'].map(lambda x: x[0])


In [11]:
data.sample(5)

,title,description_clean,genre_clean,Genre,Genre_2
3993,Prigionieri dell'oceano,several survivors of a torpedoed merchant ship...,drama war,"[drama, war]",drama
14630,Tripwire: sul filo del rasoio,a terrorist warner and an fbi agent knox fight...,action drama thriller,"[action, drama, thriller]",action
11072,L'eroe della strada,the adventures of a drifter turned illegal pri...,crime drama sport,"[crime, drama, sport]",crime
7086,Una pistola per un vile,after the death of a rancher his three sons ru...,adventure drama romance,"[adventure, drama, romance]",adventure
28086,Aakrosh,the central bureau of investigation deputes tw...,action crime thriller,"[action, crime, thriller]",action


### Split en train y test y vectorizacion de texto

In [12]:
genres = data['Genre_2'].unique()
genres

array(['romance', 'drama', 'biography', 'crime', 'western', 'fantasy',
       'comedy', 'family', 'adventure', 'action', 'mystery', 'history',
       'scifi', 'musical', 'music', 'horror', 'thriller', 'animation',
       'war', 'filmnoir', 'sport'], dtype=object)

In [13]:
from sklearn.preprocessing import OrdinalEncoder
#Cuando le paso las categorías como paramétro me tira error The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()
ord_enc = OrdinalEncoder()
data["target_genre"] = ord_enc.fit_transform(data[["Genre_2"]])


In [14]:
data.head()

,title,description_clean,genre_clean,Genre,Genre_2,target_genre
0,Miss Jerry,the adventures of a female reporter in the 1890s,romance,[romance],romance,15.0
1,Cleopatra,the fabled queen of egypts affair with roman g...,drama history,"[drama, history]",drama,6.0
2,"From the Manger to the Cross; or, Jesus of Naz...",an account of the life of jesus christ based o...,biography drama,"[biography, drama]",biography,3.0
3,Amore di madre,john howard payne at his most miserable point ...,drama,[drama],drama,6.0
4,Traffic in Souls,a woman with the aid of her police officer swe...,crime drama,"[crime, drama]",crime,5.0


In [15]:
train, test = train_test_split(data, random_state=42, test_size=0.30, shuffle=True)
train_text = train['description_clean']
test_text = test['description_clean']

In [16]:
stop_words = stopwords.words('english')
vectorizer = TfidfVectorizer(strip_accents='unicode', analyzer='word', ngram_range=(1,3), norm='l2', stop_words=stop_words)
vectorizer.fit(train_text)

TfidfVectorizer(ngram_range=(1, 3),
                stop_words=['i', 'me', 'my', 'myself', 'we', 'our', 'ours',
                            'ourselves', 'you', "you're", "you've", "you'll",
                            "you'd", 'your', 'yours', 'yourself', 'yourselves',
                            'he', 'him', 'his', 'himself', 'she', "she's",
                            'her', 'hers', 'herself', 'it', "it's", 'its',
                            'itself', ...],
                strip_accents='unicode')

In [17]:
X_train = vectorizer.transform(train_text)
X_test = vectorizer.transform(test_text)
y_train = train['target_genre']
y_test = test['target_genre']

### Pipeline de clasificación

In [18]:
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import MultinomialNB

In [ ]:
len(genres)

In [19]:
classifier_log = LogisticRegression(solver='sag')
classifier_log.fit(X_train, y_train)
prediction = classifier_log.predict(X_test)
print('Test accuracy is {}'.format(accuracy_score(y_test, prediction)))

Test accuracy is 0.4573229449356561


In [20]:
print(classification_report(y_test, prediction, zero_division=1))

              precision    recall  f1-score   support

         0.0       0.47      0.52      0.49      1932
         1.0       0.47      0.01      0.03       487
         2.0       1.00      0.00      0.00       138
         3.0       0.60      0.04      0.07       233
         4.0       0.49      0.66      0.56      3237
         5.0       0.41      0.19      0.26       833
         6.0       0.41      0.58      0.48      2747
         7.0       1.00      0.00      0.00        83
         8.0       1.00      0.00      0.00        71
         9.0       1.00      0.00      0.00        14
        10.0       1.00      0.00      0.00        10
        11.0       0.52      0.37      0.43       894
        12.0       1.00      0.00      0.00        10
        13.0       1.00      0.00      0.00        66
        14.0       1.00      0.00      0.00        90
        15.0       1.00      0.00      0.00       139
        16.0       1.00      0.00      0.00        71
        17.0       1.00    

In [21]:
from sklearn.naive_bayes import MultinomialNB

In [ ]:
# NBC=MultinomialNB();

# NBC.fit(X_train, y_train);

# test_pred = NBC.predict(X_test.todense());

In [ ]:
# print('Test accuracy is {}'.format(accuracy_score(y_test, test_pred)))

In [ ]:
# # calculating test accuracy

# print(classification_report(y_test, test_pred, zero_division=1))